In [5]:
%%writefile prefixsum_brent_kung_algorithm.cu
#define LOAD_SIZE 64
#include <iostream>
#include <cuda_runtime.h>

__global__ void prefixsum_kernel(float* A, float* C, int N) {
    int tid = threadIdx.x;
    int i = 2 * blockDim.x * blockIdx.x + tid;

    __shared__ float S_A[LOAD_SIZE];

    if (i < N) S_A[tid] = A[i];
    else S_A[tid] = 0.0f;

    if (i + blockDim.x < N) S_A[tid + blockDim.x] = A[i + blockDim.x];
    else S_A[tid + blockDim.x] = 0.0f;

    __syncthreads();

    for (int d = 1; d < LOAD_SIZE; d *= 2) {
        int j = (tid + 1) * 2 * d - 1;
        if (j < LOAD_SIZE) {
            S_A[j] += S_A[j - d];
        }
        __syncthreads();
    }

    for (int d = LOAD_SIZE / 4; d >= 1; d /= 2) {
        int j = (tid + 1) * 2 * d - 1;
        if (j + d < LOAD_SIZE) {
            S_A[j + d] += S_A[j];
        }
        __syncthreads();
    }

    if (i < N) C[i] = S_A[tid];
    if (i + blockDim.x < N) C[i + blockDim.x] = S_A[tid + blockDim.x];
}

void checkCudaError(const char* message) {
    cudaError_t error = cudaGetLastError();
    if (error != cudaSuccess) {
        printf("CUDA error (%s): %s\n", message, cudaGetErrorString(error));
        exit(-1);
    }
}

int main() {
    int N = 10;
    float A[N], C[N];

    for (int i = 0; i < N; i++) A[i] = i + 1.0f;

    float* d_A;
    float* d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    checkCudaError("Failed to copy input data to device");

    dim3 dimBlock(LOAD_SIZE / 2);
    dim3 dimGrid((N + LOAD_SIZE - 1) / LOAD_SIZE);
    prefixsum_kernel<<<dimGrid, dimBlock>>>(d_A, d_C, N);
    checkCudaError("Failed to execute the kernel");

    cudaDeviceSynchronize();
    cudaMemcpy(C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);
    checkCudaError("Failed to copy output data to host");

    cudaFree(d_A);
    cudaFree(d_C);

    printf("A:\n");
    for (int i = 0; i < N; i++) {
        printf("%.2f ", A[i]);
    }

    printf("\nC:\n");
    for (int i = 0; i < N; i++) {
        printf("%.2f ", C[i]);
    }

    return 0;
}


Writing prefixsum_brent_kung_algorithm.cu


In [6]:
# Compile with the specified architecture
!nvcc prefixsum_brent_kung_algorithm.cu -o prefixsum_brent_kung_algorithm -gencode arch=compute_75,code=sm_75

# Run the executable
!./prefixsum_brent_kung_algorithm


A:
1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 
C:
1.00 3.00 6.00 10.00 15.00 21.00 28.00 36.00 45.00 55.00 